In [1]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np
#from Stats import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList
from Training import GetDatasets
from Simulation import get_all_outputs
from Simulation import buffer_simulation, save_obj, load_obj
from funciones import compilNet, same_elements
import pandas as pd
from datetime import datetime
from pandas import ExcelWriter
from openpyxl import Workbook
from openpyxl import load_workbook
import tensorflow_datasets as tfds
#import tensorflow_probability as tfp

## Función resultados entres entradas y salidas

In [2]:
def AnlyzExp(net,Layer ):
    layers=Layer

    diff_inp_output=[]
    razon=[]
    capa=[]
    numero=[]
    sum_tensor_cociente = []
#with pd.ExcelWriter('Analizando_fichero_detalle/diferencias_entre_resultados/Alexnet_mask_0.xlsx') as writer:
    
    for i,j in enumerate(write_layer):
            
            
            print('Capa',j,net.layers[j].__class__.__name__)
            capa.append(net.layers[j].__class__.__name__)
            numero.append(j)
            #X = [x for x,y in test_dataset]
            image = next(iter(test_dataset))[0]
            #salidas del modelo sin fallas para la primer imagen del dataset de prueba
            out= get_all_outputs(net,image)
            #salidas del modelo con fallas para la primer imagen del dataset de prueba
            out_quantizacion = out[j-1]
            out_error_and_correction = out[j]
            tensor_aux= tf.math.abs(tf.math.subtract(out_quantizacion,out_error_and_correction ))
            diff_inp_out=np.sum(tf.math.abs(tf.math.subtract(out_quantizacion,out_error_and_correction )))
            input_sin_ceros=tf.math.abs(tf.where(tf.equal(out_quantizacion,0),tf.ones_like(out_quantizacion),out_quantizacion))    
            tensor_cociente=tf.math.divide(tensor_aux,input_sin_ceros)
            tensor_cociente  = tf.where(tf.less(tensor_cociente, 1), 1/tensor_cociente , tensor_cociente )
            tensor_cociente_final = tf.math.reduce_sum(tf.where(tf.math.is_inf(tensor_cociente), 0 , tensor_cociente ))
            tensor_cociente_final = np.sum(tf.where(tf.math.is_inf(tensor_cociente), 0 , tensor_cociente ))
            print('tensor_cociente final',tensor_cociente_final)
            diff_inp_output.append(diff_inp_out)
            sum_tensor_cociente.append(tensor_cociente_final)
            total=np.sum(sum_tensor_cociente)
            #print(total)
            print(sum_tensor_cociente)
            
    df_numero=pd.DataFrame(numero)
    df_capa = pd.DataFrame(capa)   
    df_inp_output=pd.DataFrame(diff_inp_output)
    df_razon=pd.DataFrame(sum_tensor_cociente)
    #print('df_razon',df_razon)
       
    buf_cociente = pd.concat([df_numero,df_capa,df_inp_output,df_razon], axis=1, join='outer')
    #print('buf_cociente',buf_cociente)
    buf_cociente.columns = ['numero','Capa','df_inp_output','df_razon']
    print(buf_cociente)
    return buf_cociente
    #with pd.ExcelWriter('Analizando_fichero_detalle/Alterado_fichero/diferencias_entre_resultados/Alexnet_mask_orig.xlsx') as writer:
    #    buf_cociente.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)
    #      

# AlexNet

In [4]:
trainBatchSize = testBatchSize = 1
_,_,test_dataset = GetDatasets('colorectal_histology',(80,5,15),(227,227), 8, trainBatchSize, testBatchSize)

C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


In [5]:
word_size  = 16
afrac_size = 11  
aint_size  = 4
wfrac_size = 11
wint_size  = 4


cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'AlexNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')

## Cambiar dirección

### Mácara Base

In [5]:
Cargar_errores = True



if Cargar_errores:
    locs  = load_obj('Data/Fault Characterization/variante_mask_vc_707/vc_707/locs_054')
    error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/vc_707/error_mask_054')
    print(len(locs))
    print(len(error_mask))

9439
9439


### Máscara volteada

In [4]:
Bese_Volt =True

if Bese_Volt:
    locs  = load_obj('Data/Fault Characterization/variante_mask_vc_707/mask_volteada/locs_054')
    error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/mask_volteada/error_mask_054')
locs[0:20]  
print(error_mask[0:20]  )

['xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x']


### Máscara en 0

In [6]:
Mask_0 =True

if Mask_0:
    locs  = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_0/vc_707/locs_054')
    error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_0/vc_707/error_mask_054')
locs[0:20]  
print(error_mask[0:20]  )

['0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000']


 ## Creo la red con el experimento original

In [7]:
from Nets_original import GetNeuralNetworkModel
from Stats import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList
NetOriginal = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                             aging_active=True, word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)



# Cuantizacion de los pesos

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
NetOriginal.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
NetOriginal.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = NetOriginal, frac_bits = afrac_size, int_bits = aint_size)
loss,acc =NetOriginal.evaluate(test_dataset)

Experimento Base
Base
750/750 [==============================] - 56s 75ms/step - loss: 0.3870 - accuracy: 0.8387


In [8]:
write_layer=[2,8,10,16,18,24,30,36,38,44,49,53]
buf_cociente=AnlyzExp(NetOriginal,write_layer)
with pd.ExcelWriter('Analizando_fichero_detalle/Alterado_fichero/diferencias_entre_resultados/Alexnet_mask_0.xlsx') as writer:
        buf_cociente.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)

Capa 2 Lambda
tensor_cociente final 1054.0
[1054.0]
Capa 8 Lambda
tensor_cociente final 2145.0
[1054.0, 2145.0]
Capa 10 Lambda
tensor_cociente final 200.0
[1054.0, 2145.0, 200.0]
Capa 16 Lambda
tensor_cociente final 1483.0
[1054.0, 2145.0, 200.0, 1483.0]
Capa 18 Lambda
tensor_cociente final 157.0
[1054.0, 2145.0, 200.0, 1483.0, 157.0]
Capa 24 Lambda
tensor_cociente final 200.0
[1054.0, 2145.0, 200.0, 1483.0, 157.0, 200.0]
Capa 30 Lambda
tensor_cociente final 200.0
[1054.0, 2145.0, 200.0, 1483.0, 157.0, 200.0, 200.0]
Capa 36 Lambda
tensor_cociente final 159.0
[1054.0, 2145.0, 200.0, 1483.0, 157.0, 200.0, 200.0, 159.0]
Capa 38 Lambda
tensor_cociente final 48.0
[1054.0, 2145.0, 200.0, 1483.0, 157.0, 200.0, 200.0, 159.0, 48.0]
Capa 44 Lambda
tensor_cociente final 0.0
[1054.0, 2145.0, 200.0, 1483.0, 157.0, 200.0, 200.0, 159.0, 48.0, 0.0]
Capa 49 Lambda
tensor_cociente final 0.0
[1054.0, 2145.0, 200.0, 1483.0, 157.0, 200.0, 200.0, 159.0, 48.0, 0.0, 0.0]
Capa 53 Lambda
tensor_cociente final 6

## Creo la red con el experimento de redondeo

In [9]:
from Nets import GetNeuralNetworkModel
from Stats_original import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList



Net2 = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=True, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
Net2.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
Net2.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = Net2, frac_bits = afrac_size, int_bits = aint_size)
loss,acc =Net2.evaluate(test_dataset)




Redondeo
Redondeo
750/750 [==============================] - 75s 100ms/step - loss: 0.3123 - accuracy: 0.8907


In [10]:
write_layer=[2,8,10,16,18,24,30,36,38,44,49,53]
buf_cociente=AnlyzExp(Net2,write_layer)
with pd.ExcelWriter('Analizando_fichero_detalle/Alterado_fichero/diferencias_entre_resultados/Alexnet_redond_volt.xlsx') as  writer:
    buf_cociente.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)

Capa 2 Lambda
tensor_cociente final 398252.53
[398252.53]
Capa 8 Lambda
tensor_cociente final 150864.5
[398252.53, 150864.5]
Capa 10 Lambda
tensor_cociente final 13897.649
[398252.53, 150864.5, 13897.649]
Capa 16 Lambda
tensor_cociente final 251854.75
[398252.53, 150864.5, 13897.649, 251854.75]
Capa 18 Lambda
tensor_cociente final 54656.582
[398252.53, 150864.5, 13897.649, 251854.75, 54656.582]
Capa 24 Lambda
tensor_cociente final 58800.367
[398252.53, 150864.5, 13897.649, 251854.75, 54656.582, 58800.367]
Capa 30 Lambda
tensor_cociente final 76679.73
[398252.53, 150864.5, 13897.649, 251854.75, 54656.582, 58800.367, 76679.73]
Capa 36 Lambda
tensor_cociente final 61009.82
[398252.53, 150864.5, 13897.649, 251854.75, 54656.582, 58800.367, 76679.73, 61009.82]
Capa 38 Lambda
tensor_cociente final 20341.4
[398252.53, 150864.5, 13897.649, 251854.75, 54656.582, 58800.367, 76679.73, 61009.82, 20341.4]
Capa 44 Lambda
tensor_cociente final 0.0
[398252.53, 150864.5, 13897.649, 251854.75, 54656.582,

## Creo la red con el experimento de Vecino

In [7]:
from NetsVecino import GetNeuralNetworkModel
from StatsVecino import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList


NetVecinos = GetNeuralNetworkModel('AlexNet', (227,227,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                             aging_active=True, word_size=word_size, frac_size=afrac_size, batch_size = testBatchSize)

#Cuantizacion de los pesos

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
NetVecinos.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
NetVecinos.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = NetVecinos, frac_bits = afrac_size, int_bits = 4)

Vecinos


In [11]:
#write_layer=[2,8,10,16,18,24,30,36,38,44,49,53]
#buf_cociente=AnlyzExp(NetVecinos,write_layer)
#with pd.ExcelWriter('Analizando_fichero_detalle/Alterado_fichero/diferencias_entre_resultados/Alexnet_vecinos.xlsx') as writer:
#        buf_cociente.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)

# Squeeznet

In [1]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np

from Training import GetDatasets
from Simulation import get_all_outputs
from Simulation import buffer_simulation, save_obj, load_obj
from funciones import compilNet, same_elements
import pandas as pd
from datetime import datetime
from pandas import ExcelWriter
from openpyxl import Workbook
from openpyxl import load_workbook
import tensorflow_datasets as tfds

trainBatchSize = testBatchSize = 1
_, _, test_dataset = GetDatasets('colorectal_histology', (80, 5, 15), (224, 224), 8, trainBatchSize, testBatchSize)


# In[3]:


# Numero de bits para activaciones (a) y pesos (w)
word_size  = 16
afrac_size = 9
aint_size  = 6
wfrac_size = 15
wint_size  = 0


cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'SqueezeNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')


C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


# Modificar dirección

In [2]:
Cargar_errores = True
if Cargar_errores:
    locs  = load_obj('Data/Fault Characterization/variante_mask_vc_707/vc_707/locs_054')
    error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/vc_707/error_mask_054')
    print(len(locs))
    print(len(error_mask))

9439
9439


### Máscara volteada

In [3]:
Bese_Volt =True

if Bese_Volt:
    locs  = load_obj('Data/Fault Characterization/variante_mask_vc_707/mask_volteada/locs_054')
    error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/mask_volteada/error_mask_054')
locs[0:20]  
print(error_mask[0:20]  )

['xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x']


### Mask_0

In [2]:
Mask_0 =True

if Mask_0:
    locs  = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_0/vc_707/locs_054')
    error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_0/vc_707/error_mask_054')
locs[0:20]  
print(error_mask[0:20]  )

['0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000']


## Red  original

In [4]:
from Nets_original import GetNeuralNetworkModel
from Stats import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList
NetSqueezeOriginal = GetNeuralNetworkModel('SqueezeNet', (224,224,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=True, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)



loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
NetSqueezeOriginal.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
NetSqueezeOriginal.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = NetSqueezeOriginal, frac_bits = afrac_size, int_bits = aint_size)
loss,acc =NetSqueezeOriginal.evaluate(test_dataset)





Experimento Base
750/750 [==============================] - 79s 106ms/step - loss: 2.4122 - accuracy: 0.7813


In [5]:
write_layer=[2,6,8,12,     19,23,     30,34,     41,43,47,     54,58,     65,69,     76,80 ,    87,89,93,    100,103,107]
buf_cociente=AnlyzExp(NetSqueezeOriginal,write_layer)
with pd.ExcelWriter('Analizando_fichero_detalle/Alterado_fichero/diferencias_entre_resultados/SqueezeNet_mask_0.xlsx') as writer:
        buf_cociente.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)

Capa 2 Lambda
tensor_cociente final 1054.0
[1054.0]
Capa 6 Lambda
tensor_cociente final 4786.0
[1054.0, 4786.0]
Capa 8 Lambda
tensor_cociente final 903.0
[1054.0, 4786.0, 903.0]
Capa 12 Lambda
tensor_cociente final 106.0
[1054.0, 4786.0, 903.0, 106.0]
Capa 19 Lambda
tensor_cociente final 2068.0
[1054.0, 4786.0, 903.0, 106.0, 2068.0]
Capa 23 Lambda
tensor_cociente final 140.0
[1054.0, 4786.0, 903.0, 106.0, 2068.0, 140.0]
Capa 30 Lambda
tensor_cociente final 1932.0
[1054.0, 4786.0, 903.0, 106.0, 2068.0, 140.0, 1932.0]
Capa 34 Lambda
tensor_cociente final 520.0
[1054.0, 4786.0, 903.0, 106.0, 2068.0, 140.0, 1932.0, 520.0]
Capa 41 Lambda
tensor_cociente final 2916.0
[1054.0, 4786.0, 903.0, 106.0, 2068.0, 140.0, 1932.0, 520.0, 2916.0]
Capa 43 Lambda
tensor_cociente final 843.0
[1054.0, 4786.0, 903.0, 106.0, 2068.0, 140.0, 1932.0, 520.0, 2916.0, 843.0]
Capa 47 Lambda
tensor_cociente final 45.0
[1054.0, 4786.0, 903.0, 106.0, 2068.0, 140.0, 1932.0, 520.0, 2916.0, 843.0, 45.0]
Capa 54 Lambda
ten

## Red con el experimento de redondeo

In [8]:
from Nets import GetNeuralNetworkModel
from Stats_original import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList
NetSqueezeRound = GetNeuralNetworkModel('SqueezeNet', (224,224,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=True, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)



loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
NetSqueezeRound.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
NetSqueezeRound.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = NetSqueezeRound, frac_bits = afrac_size, int_bits = aint_size)
loss,acc =NetSqueezeRound.evaluate(test_dataset)


Redondeo
750/750 [==============================] - 136s 181ms/step - loss: 2.5533 - accuracy: 0.7453


In [9]:
write_layer=[2,6,8,12,     19,23,     30,34,     41,43,47,     54,58,     65,69,     76,80 ,    87,89,93,    100,103,107]
buf_cociente=AnlyzExp(NetSqueezeRound,write_layer)
with pd.ExcelWriter('Analizando_fichero_detalle/Alterado_fichero/diferencias_entre_resultados/SqueezeNet_redond_volt.xlsx') as writer:
        buf_cociente.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)

Capa 2 Lambda
tensor_cociente final 94727.39
[94727.39]
Capa 6 Lambda
tensor_cociente final 520280.97
[94727.39, 520280.97]
Capa 8 Lambda
tensor_cociente final 82350.984
[94727.39, 520280.97, 82350.984]
Capa 12 Lambda
tensor_cociente final 8868.195
[94727.39, 520280.97, 82350.984, 8868.195]
Capa 19 Lambda
tensor_cociente final 139600.31
[94727.39, 520280.97, 82350.984, 8868.195, 139600.31]
Capa 23 Lambda
tensor_cociente final 6443.947
[94727.39, 520280.97, 82350.984, 8868.195, 139600.31, 6443.947]
Capa 30 Lambda
tensor_cociente final 119846.27
[94727.39, 520280.97, 82350.984, 8868.195, 139600.31, 6443.947, 119846.27]
Capa 34 Lambda
tensor_cociente final 26514.137
[94727.39, 520280.97, 82350.984, 8868.195, 139600.31, 6443.947, 119846.27, 26514.137]
Capa 41 Lambda
tensor_cociente final 348072.03
[94727.39, 520280.97, 82350.984, 8868.195, 139600.31, 6443.947, 119846.27, 26514.137, 348072.03]
Capa 43 Lambda
tensor_cociente final 55022.88
[94727.39, 520280.97, 82350.984, 8868.195, 139600.31

In [1]:
def AnlyzExp(net,Layer ):
    layers=Layer

    diff_inp_output=[]
    razon=[]
    capa=[]
    numero=[]
    sum_tensor_cociente = []
#with pd.ExcelWriter('Analizando_fichero_detalle/diferencias_entre_resultados/Alexnet_mask_0.xlsx') as writer:
    
    for i,j in enumerate(write_layer):
            
            
            print('Capa',j,net.layers[j].__class__.__name__)
            capa.append(net.layers[j].__class__.__name__)
            numero.append(j)
            #X = [x for x,y in test_dataset]
            image = next(iter(test_dataset))[0]
            #salidas del modelo sin fallas para la primer imagen del dataset de prueba
            out= get_all_outputs(net,image)
            #salidas del modelo con fallas para la primer imagen del dataset de prueba
            out_quantizacion = out[j-1]
            out_error_and_correction = out[j]
            tensor_aux= tf.math.abs(tf.math.subtract(out_quantizacion,out_error_and_correction ))
            diff_inp_out=np.sum(tf.math.abs(tf.math.subtract(out_quantizacion,out_error_and_correction )))
            input_sin_ceros=tf.math.abs(tf.where(tf.equal(out_quantizacion,0),tf.ones_like(out_quantizacion),out_quantizacion))    
            tensor_cociente=tf.math.divide(tensor_aux,input_sin_ceros)
            tensor_cociente  = tf.where(tf.less(tensor_cociente, 1), 1/tensor_cociente , tensor_cociente )
            tensor_cociente_final = tf.math.reduce_sum(tf.where(tf.math.is_inf(tensor_cociente), 0 , tensor_cociente ))
            tensor_cociente_final = np.sum(tf.where(tf.math.is_inf(tensor_cociente), 0 , tensor_cociente ))
            print('tensor_cociente final',tensor_cociente_final)
            diff_inp_output.append(diff_inp_out)
            sum_tensor_cociente.append(tensor_cociente_final)
            total=np.sum(sum_tensor_cociente)
            #print(total)
            print(sum_tensor_cociente)
            
    df_numero=pd.DataFrame(numero)
    df_capa = pd.DataFrame(capa)   
    df_inp_output=pd.DataFrame(diff_inp_output)
    df_razon=pd.DataFrame(sum_tensor_cociente)
    #print('df_razon',df_razon)
       
    buf_cociente = pd.concat([df_numero,df_capa,df_inp_output,df_razon], axis=1, join='outer')
    #print('buf_cociente',buf_cociente)
    buf_cociente.columns = ['numero','Capa','df_inp_output','df_razon']
    print(buf_cociente)
    return buf_cociente
    #with pd.ExcelWriter('Analizando_fichero_detalle/Alterado_fichero/diferencias_entre_resultados/Alexnet_mask_orig.xlsx') as writer:
    #    buf_cociente.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)
    #      

## MobileNet

In [2]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np
from Training import GetDatasets
from Simulation import get_all_outputs
from Simulation import buffer_simulation, save_obj, load_obj
from funciones import compilNet, same_elements
import pandas as pd
from datetime import datetime
from pandas import ExcelWriter
from openpyxl import Workbook
from openpyxl import load_workbook
import tensorflow_datasets as tfds

trainBatchSize = testBatchSize = 1
_, _, test_dataset = GetDatasets('colorectal_histology', (80, 5, 15), (224, 224), 8, trainBatchSize, testBatchSize)


# In[3]:


# Numero de bits para activaciones (a) y pesos (w)
word_size  = 16
afrac_size = 11
aint_size  = 4
wfrac_size = 14
wint_size  = 1

# Tamaño del buffer de activaciones == al tamaño de la capa con mayor numero de activaciones (290400 pesos de 16 bits cada uno)
abuffer_size = 16777216
# Directorio de los pesos
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'MobileNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')

Cargar_errores = True





layer=[2,9,15,21,28,34,40,46,53,59,65,71,78,84,90,96,102,108,114,120,126,132,138,144,151,157,163,169,173,179]

C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


In [3]:
Bese_Volt =True

if Bese_Volt:
    locs  = load_obj('Data/Fault Characterization/variante_mask_vc_707/mask_volteada/locs_054')
    error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/mask_volteada/error_mask_054')
locs[0:20]  
print(error_mask[0:20]  )

['xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx0x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx1xxxxxx', 'xxxxxxxxxxxxxx1x', 'xxxxxxxxx0xxxxxx', 'xxxxxxxxxxxxxx0x']


In [3]:
Mask_0 =True

if Mask_0:
    locs  = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_0/vc_707/locs_054')
    error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/error_mask_0/vc_707/error_mask_054')
locs[0:20]  
print(error_mask[0:20]  )

['0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000', '0000000000000000']


In [4]:
from Nets_original import GetNeuralNetworkModel
from Stats import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList
MobileNetOriginal = GetNeuralNetworkModel('MobileNet', (224,224,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=True, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
MobileNetOriginal.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
MobileNetOriginal.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = MobileNetOriginal, frac_bits = afrac_size, int_bits = aint_size)
loss,acc =MobileNetOriginal.evaluate(test_dataset)





Experimento Base
750/750 [==============================] - 121s 161ms/step - loss: 2.8904 - accuracy: 0.4347


In [5]:
write_layer=[2,9,15,21,28,34,40,46,53,59,65,71,78,84,90,96,102,108,114,120,126,132,138,144,151,157,163,169,173,179]
buf_cociente=AnlyzExp(MobileNetOriginal,write_layer)
with pd.ExcelWriter('Analizando_fichero_detalle/Alterado_fichero/diferencias_entre_resultados/MobileNet_mask_0.xlsx') as writer:
        buf_cociente.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)

Capa 2 Lambda
tensor_cociente final 1054.0
[1054.0]
Capa 9 Lambda
tensor_cociente final 2487.0
[1054.0, 2487.0]
Capa 15 Lambda
tensor_cociente final 1696.0
[1054.0, 2487.0, 1696.0]
Capa 21 Lambda
tensor_cociente final 4093.0
[1054.0, 2487.0, 1696.0, 4093.0]
Capa 28 Lambda
tensor_cociente final 777.0
[1054.0, 2487.0, 1696.0, 4093.0, 777.0]
Capa 34 Lambda
tensor_cociente final 1622.0
[1054.0, 2487.0, 1696.0, 4093.0, 777.0, 1622.0]
Capa 40 Lambda
tensor_cociente final 1837.0
[1054.0, 2487.0, 1696.0, 4093.0, 777.0, 1622.0, 1837.0]
Capa 46 Lambda
tensor_cociente final 1498.0
[1054.0, 2487.0, 1696.0, 4093.0, 777.0, 1622.0, 1837.0, 1498.0]
Capa 53 Lambda
tensor_cociente final 334.0
[1054.0, 2487.0, 1696.0, 4093.0, 777.0, 1622.0, 1837.0, 1498.0, 334.0]
Capa 59 Lambda
tensor_cociente final 690.0
[1054.0, 2487.0, 1696.0, 4093.0, 777.0, 1622.0, 1837.0, 1498.0, 334.0, 690.0]
Capa 65 Lambda
tensor_cociente final 507.0
[1054.0, 2487.0, 1696.0, 4093.0, 777.0, 1622.0, 1837.0, 1498.0, 334.0, 690.0, 507

In [5]:
from Nets import GetNeuralNetworkModel
from Stats_original import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList
MobileNetRound = GetNeuralNetworkModel('MobileNet', (224,224,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=True, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
MobileNetRound.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
MobileNetRound.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = MobileNetRound, frac_bits = afrac_size, int_bits = aint_size)
loss,acc =MobileNetRound.evaluate(test_dataset)



Redondeo
750/750 [==============================] - 219s 292ms/step - loss: 0.3685 - accuracy: 0.8707


In [6]:
write_layer=[2,9,15,21,28,34,40,46,53,59,65,71,78,84,90,96,102,108,114,120,126,132,138,144,151,157,163,169,173,179]
buf_cociente=AnlyzExp(MobileNetRound,write_layer)
with pd.ExcelWriter('Analizando_fichero_detalle/Alterado_fichero/diferencias_entre_resultados/MobileNet_redond_volt.xlsx') as writer:
        buf_cociente.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)

Capa 2 Lambda
tensor_cociente final 398495.8
[398495.8]
Capa 9 Lambda
tensor_cociente final 1099181.9
[398495.8, 1099181.9]
Capa 15 Lambda
tensor_cociente final 832428.2
[398495.8, 1099181.9, 832428.2]
Capa 21 Lambda
tensor_cociente final 2433053.8
[398495.8, 1099181.9, 832428.2, 2433053.8]
Capa 28 Lambda
tensor_cociente final 299521.2
[398495.8, 1099181.9, 832428.2, 2433053.8, 299521.2]
Capa 34 Lambda
tensor_cociente final 761243.8
[398495.8, 1099181.9, 832428.2, 2433053.8, 299521.2, 761243.8]
Capa 40 Lambda
tensor_cociente final 648331.1
[398495.8, 1099181.9, 832428.2, 2433053.8, 299521.2, 761243.8, 648331.1]
Capa 46 Lambda
tensor_cociente final 694540.6
[398495.8, 1099181.9, 832428.2, 2433053.8, 299521.2, 761243.8, 648331.1, 694540.6]
Capa 53 Lambda
tensor_cociente final 98651.18
[398495.8, 1099181.9, 832428.2, 2433053.8, 299521.2, 761243.8, 648331.1, 694540.6, 98651.18]
Capa 59 Lambda
tensor_cociente final 339045.72
[398495.8, 1099181.9, 832428.2, 2433053.8, 299521.2, 761243.8, 648

## VGG16

In [2]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np
from Training import GetDatasets
from Simulation import get_all_outputs
from Simulation import buffer_simulation, save_obj, load_obj
from funciones import compilNet, same_elements
import pandas as pd
from datetime import datetime
from pandas import ExcelWriter
from openpyxl import Workbook
from openpyxl import load_workbook
import tensorflow_datasets as tfds



trainBatchSize = testBatchSize = 1
_, _, test_dataset = GetDatasets('colorectal_histology', (80, 5, 15), (224, 224), 8, trainBatchSize, testBatchSize)


word_size  = 16
afrac_size = 12
aint_size  = 3
wfrac_size = 15
wint_size  = 0

# Tamaño del buffer de activaciones == al tamaño de la capa con mayor numero de activaciones (290400 pesos de 16 bits cada uno)
abuffer_size = 16777216
# Directorio de los pesos
cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'VGG16')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')





C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


In [4]:
from Nets_original import GetNeuralNetworkModel
from Stats import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList
VGG16Original = GetNeuralNetworkModel('VGG16', (224,224,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=True, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
VGG16Original.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
VGG16Original.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = VGG16Original, frac_bits = afrac_size, int_bits = aint_size)
loss,acc =VGG16Original.evaluate(test_dataset)

Experimento Base
750/750 [==============================] - 482s 642ms/step - loss: 1.1234 - accuracy: 0.6360


In [5]:
write_layer=[2,6,10,12,16,20,22,26,30,34,36,40,44,48,50,54,58,62,64,69,73,77]
buf_cociente=AnlyzExp(VGG16Original,write_layer)
with pd.ExcelWriter('Analizando_fichero_detalle/Alterado_fichero/diferencias_entre_resultados/VGG16_mask_0.xlsx') as writer:
        buf_cociente.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)

Capa 2 Lambda
tensor_cociente final 1054.0
[1054.0]
Capa 6 Lambda
tensor_cociente final 5423.0
[1054.0, 5423.0]
Capa 10 Lambda
tensor_cociente final 4314.0
[1054.0, 5423.0, 4314.0]
Capa 12 Lambda
tensor_cociente final 3855.0
[1054.0, 5423.0, 4314.0, 3855.0]
Capa 16 Lambda
tensor_cociente final 2701.0
[1054.0, 5423.0, 4314.0, 3855.0, 2701.0]
Capa 20 Lambda
tensor_cociente final 2379.0
[1054.0, 5423.0, 4314.0, 3855.0, 2701.0, 2379.0]
Capa 22 Lambda
tensor_cociente final 1540.0
[1054.0, 5423.0, 4314.0, 3855.0, 2701.0, 2379.0, 1540.0]
Capa 26 Lambda
tensor_cociente final 2368.0
[1054.0, 5423.0, 4314.0, 3855.0, 2701.0, 2379.0, 1540.0, 2368.0]
Capa 30 Lambda
tensor_cociente final 1834.0
[1054.0, 5423.0, 4314.0, 3855.0, 2701.0, 2379.0, 1540.0, 2368.0, 1834.0]
Capa 34 Lambda
tensor_cociente final 1471.0
[1054.0, 5423.0, 4314.0, 3855.0, 2701.0, 2379.0, 1540.0, 2368.0, 1834.0, 1471.0]
Capa 36 Lambda
tensor_cociente final 167.0
[1054.0, 5423.0, 4314.0, 3855.0, 2701.0, 2379.0, 1540.0, 2368.0, 1834

In [6]:
from Nets import GetNeuralNetworkModel
from Stats_original import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList
VGG16Round = GetNeuralNetworkModel('VGG16', (224,224,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=True, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
VGG16Round.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
VGG16Round.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = VGG16Round, frac_bits = afrac_size, int_bits = aint_size)
#loss,acc =VGG16Round.evaluate(test_dataset)

Redondeo


In [7]:
write_layer=[2,6,10,12,16,20,22,26,30,34,36,40,44,48,50,54,58,62,64,69,73,77]
buf_cociente=AnlyzExp(VGG16Round,write_layer)
with pd.ExcelWriter('Analizando_fichero_detalle/Alterado_fichero/diferencias_entre_resultados/VGG16_redond_volt.xlsx') as writer:
        buf_cociente.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)

Capa 2 Lambda
tensor_cociente final 755968.25
[755968.25]
Capa 6 Lambda
tensor_cociente final 2554692.2
[755968.25, 2554692.2]
Capa 10 Lambda
tensor_cociente final 2648391.0
[755968.25, 2554692.2, 2648391.0]
Capa 12 Lambda
tensor_cociente final 1887625.0
[755968.25, 2554692.2, 2648391.0, 1887625.0]
Capa 16 Lambda
tensor_cociente final 3399579.5
[755968.25, 2554692.2, 2648391.0, 1887625.0, 3399579.5]
Capa 20 Lambda
tensor_cociente final 3597531.2
[755968.25, 2554692.2, 2648391.0, 1887625.0, 3399579.5, 3597531.2]
Capa 22 Lambda
tensor_cociente final 855786.75
[755968.25, 2554692.2, 2648391.0, 1887625.0, 3399579.5, 3597531.2, 855786.75]
Capa 26 Lambda
tensor_cociente final 2644984.5
[755968.25, 2554692.2, 2648391.0, 1887625.0, 3399579.5, 3597531.2, 855786.75, 2644984.5]
Capa 30 Lambda
tensor_cociente final 2865987.8
[755968.25, 2554692.2, 2648391.0, 1887625.0, 3399579.5, 3597531.2, 855786.75, 2644984.5, 2865987.8]
Capa 34 Lambda
tensor_cociente final 2934713.2
[755968.25, 2554692.2, 26483

## ZFNet

In [1]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np
from Training import GetDatasets
from Simulation import get_all_outputs
from Simulation import buffer_simulation, save_obj, load_obj
from funciones import compilNet, same_elements
import pandas as pd
from datetime import datetime
from pandas import ExcelWriter
from openpyxl import Workbook
from openpyxl import load_workbook
import tensorflow_datasets as tfds




trainBatchSize = testBatchSize = 1
_, _, test_dataset = GetDatasets('colorectal_histology', (80, 5, 15), (224, 224), 8, trainBatchSize, testBatchSize)





word_size  = 16
afrac_size = 11
aint_size  = 4
wfrac_size = 14
wint_size  = 1


cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'ZFNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')



C:\Users\usuario\anaconda3\envs\env_first\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:3703: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  warnings.warn(


9439
9439


In [3]:
from Nets_original import GetNeuralNetworkModel
from Stats import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList
ZFNetOriginal = GetNeuralNetworkModel('ZFNet', (224,224,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=True, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
ZFNetOriginal.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
ZFNetOriginal.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = ZFNetOriginal, frac_bits = afrac_size, int_bits = aint_size)
loss,acc =ZFNetOriginal.evaluate(test_dataset)

750/750 [==============================] - 87s 116ms/step - loss: 12.4162 - accuracy: 0.1173


In [6]:
write_layer=[2,6,10,14,18,22,26,30,32,36,39,43]
buf_cociente=AnlyzExp(ZFNetOriginal,write_layer)
with pd.ExcelWriter('Analizando_fichero_detalle/Alterado_fichero/diferencias_entre_resultados/ZFNet_mask_orig.xlsx') as writer:
        buf_cociente.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)

Capa 2 Lambda
tensor_cociente final 122488.336
[122488.336]
Capa 6 Lambda
tensor_cociente final 499913.88
[122488.336, 499913.88]
Capa 10 Lambda
tensor_cociente final 144222.84
[122488.336, 499913.88, 144222.84]
Capa 14 Lambda
tensor_cociente final 115019.72
[122488.336, 499913.88, 144222.84, 115019.72]
Capa 18 Lambda
tensor_cociente final 15944.09
[122488.336, 499913.88, 144222.84, 115019.72, 15944.09]
Capa 22 Lambda
tensor_cociente final 17239.438
[122488.336, 499913.88, 144222.84, 115019.72, 15944.09, 17239.438]
Capa 26 Lambda
tensor_cociente final 13178.362
[122488.336, 499913.88, 144222.84, 115019.72, 15944.09, 17239.438, 13178.362]
Capa 30 Lambda
tensor_cociente final 12283.836
[122488.336, 499913.88, 144222.84, 115019.72, 15944.09, 17239.438, 13178.362, 12283.836]
Capa 32 Lambda
tensor_cociente final 2774.239
[122488.336, 499913.88, 144222.84, 115019.72, 15944.09, 17239.438, 13178.362, 12283.836, 2774.239]
Capa 36 Lambda
tensor_cociente final 0.0
[122488.336, 499913.88, 144222.8

In [7]:
from Nets import GetNeuralNetworkModel
from Stats_original import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList
ZFNetRound = GetNeuralNetworkModel('ZFNet', (224,224,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=True, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
ZFNetRound.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
ZFNetRound.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = ZFNetRound, frac_bits = afrac_size, int_bits = aint_size)
loss,acc =ZFNetRound.evaluate(test_dataset)

Redondeo
750/750 [==============================] - 122s 162ms/step - loss: 12.4162 - accuracy: 0.1173


In [8]:
write_layer=[2,6,10,14,18,22,26,30,32,36,39,43]
buf_cociente=AnlyzExp(ZFNetRound,write_layer)
with pd.ExcelWriter('Analizando_fichero_detalle/Alterado_fichero/diferencias_entre_resultados/ZFNet_mask_redond.xlsx') as writer:
        buf_cociente.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)

Capa 2 Lambda
tensor_cociente final 193054.28
[193054.28]
Capa 6 Lambda
tensor_cociente final 529806.9
[193054.28, 529806.9]
Capa 10 Lambda
tensor_cociente final 128657.555
[193054.28, 529806.9, 128657.555]
Capa 14 Lambda
tensor_cociente final 133133.42
[193054.28, 529806.9, 128657.555, 133133.42]
Capa 18 Lambda
tensor_cociente final 10003.197
[193054.28, 529806.9, 128657.555, 133133.42, 10003.197]
Capa 22 Lambda
tensor_cociente final 22091.963
[193054.28, 529806.9, 128657.555, 133133.42, 10003.197, 22091.963]
Capa 26 Lambda
tensor_cociente final 16392.47
[193054.28, 529806.9, 128657.555, 133133.42, 10003.197, 22091.963, 16392.47]
Capa 30 Lambda
tensor_cociente final 12100.453
[193054.28, 529806.9, 128657.555, 133133.42, 10003.197, 22091.963, 16392.47, 12100.453]
Capa 32 Lambda
tensor_cociente final 2758.854
[193054.28, 529806.9, 128657.555, 133133.42, 10003.197, 22091.963, 16392.47, 12100.453, 2758.854]
Capa 36 Lambda
tensor_cociente final 0.0
[193054.28, 529806.9, 128657.555, 133133.

# DenseNet

In [2]:
import os
import pickle as pickle
import tensorflow as tf
import numpy as np
from Training import GetDatasets
from Simulation import get_all_outputs
from Simulation import buffer_simulation, save_obj, load_obj
from datetime import datetime
import pandas as pd
from funciones import compilNet, same_elements




trainBatchSize = testBatchSize = 1
_, _, test_dataset = GetDatasets('colorectal_histology', (80, 5, 15), (224, 224), 8, trainBatchSize, testBatchSize)



word_size  = 16
afrac_size = 12
aint_size  = 3
wfrac_size = 13
wint_size  = 2


cwd = os.getcwd()
wgt_dir = os.path.join(cwd, 'Data')
wgt_dir = os.path.join(wgt_dir, 'Trained Weights')
wgt_dir = os.path.join(wgt_dir, 'DenseNet')
wgt_dir = os.path.join(wgt_dir, 'Colorectal Dataset')
wgt_dir = os.path.join(wgt_dir,'Weights')



In [3]:
Cargar_errores = True
if Cargar_errores:
    locs  = load_obj('Data/Fault Characterization/variante_mask_vc_707/vc_707/locs_054')
    error_mask = load_obj('Data/Fault Characterization/variante_mask_vc_707/vc_707/error_mask_054')
    print(len(locs))
    print(len(error_mask))

9439
9439


In [5]:
from Nets_original import GetNeuralNetworkModel
from Stats import WeightQuantization, IntroduceFaultsInWeights, GenerateFaultsList
DenseNetOriginal = GetNeuralNetworkModel('DenseNet', (224,224,3), 8, faulty_addresses=locs, masked_faults=error_mask,
                                 aging_active=True, word_size=word_size, frac_size=afrac_size, 
                                 batch_size = testBatchSize)
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
DenseNetOriginal.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
DenseNetOriginal.load_weights(wgt_dir).expect_partial()
WeightQuantization(model = DenseNetOriginal, frac_bits = afrac_size, int_bits = aint_size)
loss,acc =DenseNetOriginal.evaluate(test_dataset)

750/750 [==============================] - 452s 603ms/step - loss: 0.7139 - accuracy: 0.6600


In [8]:
write_layer=[2,9,11,15,21,(23,11),28,34,(36,24),41,47,(49,37),54,60,(62,50),67,73,(75,63),80,86,(88,76),93,96,98,102,108,(110,98),
     115,121,(123,111),128,134,(136,124),141,147,(149,137),154,160,(162,150),167,173,(175,163),180,186,(188,176),
     193,199,(201,189),206,212,(214,202),219,225,(227,215),232,238,(240,228),245,251,(253,241),258,261,263,267,273,(275,263),
     280,286,(288,276),293,299,(301,289),306,312,(314,302),319,325,(327,315),332,338,(340,328),345,351,(353,341),
     358,364,(366,353),371,377,(379,367),384,390,(392,380),397,403,(405,393),410,416,(418,406),423,429,(431,419),
     436,442,(444,432),449,455,(457,445),462,468,(470,458),475,481,(483,471),488,494,(496,484),501,507,(509,497),
     514,520,(522,510),527,533,(535,523),540,546,(548,536),553,559,(561,549),566,572,(574,562),579,582,584,588,594,(596,584),
     601,607,(609,597),614,620,(622,610),627,633,(635,623),640,646,(648,636),653,659,(661,649),666,672,(674,662),
     679,685,(687,675),692,698,(700,688),705,711,(713,701),718,724,(726,714),731,737,(739,727),744,750,(752,740),
     757,763,(765,753),770,776,(778,766),783,789,(791,779),796,799,803]
buf_cociente=AnlyzExp(DenseNetOriginal,write_layer)
with pd.ExcelWriter('Analizando_fichero_detalle/Alterado_fichero/diferencias_entre_resultados/DenseNet_mask_orig.xlsx') as writer:
        buf_cociente.to_excel(writer, sheet_name='buf_diff_inp_out', index=False)

Capa 2 Lambda
tensor_cociente final 249138.75
[249138.75]
Capa 9 Lambda
tensor_cociente final 2083457.5
[249138.75, 2083457.5]
Capa 11 MaxPooling2D


InvalidArgumentError: Incompatible shapes: [1,114,114,64] vs. [1,56,56,64] [Op:Sub]